In [4]:
import pandas as pd
data = pd.read_csv('Downloads/creditcard.csv')

In [5]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
#Normalizing features except for time and amount
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.iloc[:, 1:-2] = scaler.fit_transform(data.iloc[:, 1:-2])

In [7]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,149.62,0
1,0.0,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,2.69,0
2,1.0,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,378.66,0
3,1.0,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,123.50,0
4,2.0,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,69.99,0


In [8]:
X = data.drop(columns=['Class']).values
y = data['Class'].values

In [9]:
#Convert to pytorch tensors
import torch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [24]:
#Transformer based Autoencoder
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, dropout):
        super(TransformerAutoencoder, self).__init__()
        encoder_layers = TransformerEncoderLayer(d_model=input_dim, nhead=n_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True)
        self.encoder = TransformerEncoder(encoder_layers, num_layers=n_layers)
        self.decoder = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [14]:
#Contrastive Learning
def contrastive_loss(x1, x2, temperature=0.5):
    x1_normalized = F.normalize(x1, dim=-1)
    x2_normalized = F.normalize(x2, dim=-1)
    return 2 - 2 * (x1_normalized * x2_normalized).sum(dim=-1)

In [15]:
#Generative Adversarial Networks (GAN): Not needed for this dataset
#GANs require a large amount of data to effectively learn the underlying data distribution and generate realistic samples. In anomaly detection tasks, the amount of anomalous data is often limited, making it challenging to train a GAN effectively.

In [26]:
#Training and Evaluation
input_dim = X.shape[1]
hidden_dim = 128
n_layers = 2
n_heads = 2
dropout = 0.1
batch_size = 32
epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
length = X.shape[1]
print("Length of the second dimension:", length)

Length of the second dimension: 30


In [27]:
#Define dataset and dataloader
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(X_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [28]:
#Initialize model, optimizer, and loss function
import torch.optim as optim
model = TransformerAutoencoder(input_dim, hidden_dim, n_layers, n_heads, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
#Train the model
import torch.nn.functional as F
from tqdm import tqdm
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        batch = batch[0].to(device)
        optimizer.zero_grad()
        output = model(batch)
        loss = F.mse_loss(output, batch)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}")

Epoch 1/10: 100%|██████████████████████████████████████████████████████████████████| 8901/8901 [02:55<00:00, 50.73it/s]


Epoch 1/10, Loss: 357706511.7806988


Epoch 2/10: 100%|██████████████████████████████████████████████████████████████████| 8901/8901 [03:01<00:00, 49.02it/s]


Epoch 2/10, Loss: 285286398.3642287


Epoch 3/10: 100%|██████████████████████████████████████████████████████████████████| 8901/8901 [02:58<00:00, 49.92it/s]


...

Epoch 8/10, Loss: 65854483.23064824


Epoch 9/10: 100%|██████████████████████████████████████████████████████████████████| 8901/8901 [01:33<00:00, 95.51it/s]


Epoch 9/10, Loss: 64572123.44320863


Epoch 10/10: 100%|█████████████████████████████████████████████████████████████████| 8901/8901 [01:29<00:00, 99.92it/s]

Epoch 10/10, Loss: 63859160.14335468


In [31]:
#Evaluation
import numpy as np
model.eval()
with torch.no_grad():
    reconstructions = []
    for batch in dataloader:
        batch = batch[0].to(device)
        recon = model(batch)
        reconstructions.append(recon.cpu().numpy())

reconstructions = np.concatenate(reconstructions, axis=0)
mse = np.mean(np.power(X - reconstructions, 2), axis=1)
threshold = np.percentile(mse, 95)  # Adjust threshold as per requirement

In [32]:
anomalies = data[mse > threshold]
print("Detected anomalies:", anomalies)

Detected anomalies:             Time        V1        V2        V3        V4        V5        V6  \
0            0.0 -0.694242 -0.044075  1.672773  0.973366 -0.245117  0.347068   
1            0.0  0.608496  0.161176  0.109797  0.316523  0.043483 -0.061820   
2            1.0 -0.693500 -0.811578  1.169468  0.268231 -0.364572  1.351454   
3            1.0 -0.493325 -0.112169  1.182516 -0.609727 -0.007469  0.936150   
4            2.0 -0.591330  0.531541  1.021412  0.284655 -0.295015  0.071999   
...          ...       ...       ...       ...       ...       ...       ...   
284795  172778.0 -6.390351  6.169553 -5.590541 -1.773101 -3.323085 -1.046684   
284796  172780.0  0.962300 -0.086925 -0.659483  1.064205 -0.025575 -0.460596   
284797  172782.0 -0.123512  0.431323  0.263680 -0.327295  0.177165 -1.008556   
284798  172782.0  0.112079  0.533666 -0.419383  0.678686 -0.110829 -0.761338   
284799  172783.0 -0.906286 -0.002565  0.784689  0.233847  0.866559  4.143293   

              V7   

In [33]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,149.62,0
1,0.0,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,2.69,0
2,1.0,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,378.66,0
3,1.0,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,123.50,0
4,2.0,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-6.065842,6.099286,-6.486245,-1.459641,-3.886611,-1.956690,-3.975628,6.116573,1.742559,...,0.290602,0.154146,1.624574,-0.841000,2.756320,0.518500,2.337901,2.495529,0.77,0
284803,172787.0,-0.374121,-0.033356,1.342145,-0.521651,0.629040,0.794446,0.019667,0.246886,0.532299,...,0.291625,1.273781,0.019958,-1.677920,-1.163726,-0.819647,0.169641,-0.162164,24.79,0
284804,172788.0,0.980024,-0.182434,-2.143205,-0.393984,1.905833,2.275262,-0.239939,0.593140,0.393630,...,0.315913,0.796788,-0.060053,1.056944,0.509797,-0.181182,0.011037,-0.080467,67.88,0
284805,172788.0,-0.122755,0.321250,0.463320,0.487192,-0.273836,0.468155,-0.554672,0.568631,0.356887,...,0.361112,1.102451,-0.261503,0.203428,-1.091855,1.133635,0.269604,0.316687,10.00,0
